In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
train_path = '../dataset/mnist/MNIST/processed/training.pt'
test_path = '../dataset/mnist/MNIST/processed/test.pt'

In [16]:
class Data(Dataset):
    def __init__(self, path):
        self.features, self.labels = torch.load(path)
        self.len = self.features.size(0)
        self.features = (self.features.float()/255).view(-1,1,28,28)
        
    def __getitem__(self,index):
        return self.features[index], self.labels[index]
        
    def __len__(self):
        return self.len

In [17]:
train_dataset = Data(train_path)
test_dataset = Data(test_path)
train_loader = DataLoader(dataset = train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size=64, shuffle=True)

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5, stride = 1,padding=2)
        self.conv2 = nn.Conv2d(10,20,kernel_size=5, stride = 1, padding=2)
        self.mp = nn.MaxPool2d(2)
        self.l1 = nn.Linear(980, 192)
        self.l2 = nn.Linear(192, 10)
        
    def forward(self,x):
        batch_size= x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.mp(F.relu(self.conv2(x)))
        x = x.view(batch_size, -1)
        x = F.relu(self.l1(x))
        x = F.softmax(self.l2(x))
        return x
    

In [19]:
model = CNN().to(device)

In [20]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [21]:
def train(epoch):
    cost = 0
    for i in range(epoch):
        for index, data in enumerate(train_loader):
            optimizer.zero_grad()
            # x.size = [batch_size, -1, 28, 28]
            # y.size = [batch_size]
            x, y = data
            x = x.to(device)
            y = y.to(device)
            #output.size(batch_size, 10)
            output = model(x)
            l = loss(output, y)
            cost += l.item()
            l.backward()
            optimizer.step()
            if index%300 == 299:
                loss_ave = cost/300
                print('[%5d, %5d] loss : %5f' % (i+1, index+1, loss_ave))
                cost = 0

In [22]:
def test():
    total = 0
    correct = 0
    ls = 0
    idx = 0
    for index, (x, y) in enumerate(test_loader):
        x = x.to(device)
        y = y.to(device)
        predictions = model(x)
        l = loss(predictions, y)
        ls += l.item()
        #predictio
        p = torch.max(predictions, dim=1)
        total = y.size(0)
        correct = torch.eq(p.indices, y).sum().item()
        idx = index
    acc_test = correct / total * 100
    print('Accuracy on test set : %.3f %%' % (acc_test))
    loss_test = ls / idx
    return acc_test, loss_test

In [23]:
train(5)

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


[    1,   300] loss : 2.234765
[    1,   600] loss : 1.733368
[    1,   900] loss : 1.638447
[    2,   300] loss : 1.817114
[    2,   600] loss : 1.603620
[    2,   900] loss : 1.570740
[    3,   300] loss : 1.703996
[    3,   600] loss : 1.505391
[    3,   900] loss : 1.498169
[    4,   300] loss : 1.682638
[    4,   600] loss : 1.493220
[    4,   900] loss : 1.491034
[    5,   300] loss : 1.678332
[    5,   600] loss : 1.484697
[    5,   900] loss : 1.487667


In [24]:
test()

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Accuracy on test set : 93.750 %


(93.75, 1.4944715927808712)